<a href="https://colab.research.google.com/github/divyansh-k12/MP-iris-isef/blob/main/DigiTwinOSs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# digestive system
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ============================================================
# 1. Particle classes and representative size ranges
# ============================================================
MP_CLASSES = ["Nano", "Micro", "Macro"]

SIZE_RANGES = {
    "Nano": "<1 µm (true nanoplastics)",
    "Micro": "1–100 µm",
    "Macro": "100 µm – 1–4 mm"
}

# Initial ingestion (particles swallowed)
INITIAL = {"Nano": 1000, "Micro": 700, "Macro": 300}

# ============================================================
# 2. Compartments of the digestive system
# ============================================================
COMPARTMENTS = [
    "Stomach_Lumen",
    "Intestine_Lumen",
    "Mucus_Layer",
    "Epithelium",
    "Blood",
    "Liver",
    "Fecal_Excretion"
]

# ============================================================
# 3. Rate parameters (per 0.5-hour timestep)
# Added detachment terms (reverse fluxes):
# - mucus_detach: Mucus → Intestine_Lumen
# - epith_detach: Epithelium → Mucus_Layer
# - liver_detach: Liver → Blood
# ============================================================
RATES = {
    "Nano": {
        "gastric_empty": 0.25,   # stomach → intestine
        "mucus_trap": 0.05,      # intestine lumen → mucus
        "mucus_detach": 0.04,    # mucus → lumen
        "absorb_epith": 0.02,    # lumen/mucus → epithelium
        "epith_detach": 0.01,    # epithelium → mucus
        "enter_blood": 0.01,     # epithelium → blood
        "to_liver": 0.6,         # blood → liver
        "liver_detach": 0.02,    # liver → blood
        "fecal_loss": 0.01,      # lumen → excretion
        "hepatic_clear": 0.03    # liver → bile/excretion
    },
    "Micro": {
        "gastric_empty": 0.35,
        "mucus_trap": 0.07,
        "mucus_detach": 0.02,
        "absorb_epith": 0.002,
        "epith_detach": 0.001,
        "enter_blood": 0.0005,
        "to_liver": 0.7,
        "liver_detach": 0.01,
        "fecal_loss": 0.03,
        "hepatic_clear": 0.02
    },
    "Macro": {
        "gastric_empty": 0.40,
        "mucus_trap": 0.10,
        "mucus_detach": 0.005,
        "absorb_epith": 0.0001,
        "epith_detach": 0.00005,
        "enter_blood": 0.00001,
        "to_liver": 0.8,
        "liver_detach": 0.002,
        "fecal_loss": 0.05,
        "hepatic_clear": 0.01
    }
}

# ============================================================
# 4. Simulation configuration
# ============================================================
total_hours = 48
dt_hours = 0.5
steps = int(total_hours / dt_hours)

# Initialize state
state = {cls: {c: 0.0 for c in COMPARTMENTS} for cls in MP_CLASSES}
for cls in MP_CLASSES:
    state[cls]["Stomach_Lumen"] = INITIAL[cls]

history = []

# ============================================================
# 5. Simulation loop
# ============================================================
for step in range(steps + 1):
    snapshot = {cls: state[cls].copy() for cls in MP_CLASSES}
    history.append(snapshot)

    new_state = {cls: {c: 0.0 for c in COMPARTMENTS} for cls in MP_CLASSES}

    for cls in MP_CLASSES:
        p = RATES[cls]
        s = state[cls]

        # --- Stomach dynamics ---
        move_to_intestine = s["Stomach_Lumen"] * p["gastric_empty"]
        remain_stomach = s["Stomach_Lumen"] - move_to_intestine

        # --- Intestine lumen dynamics ---
        mucus_trap = s["Intestine_Lumen"] * p["mucus_trap"]
        absorb_epith = s["Intestine_Lumen"] * p["absorb_epith"]
        fecal_loss = s["Intestine_Lumen"] * p["fecal_loss"]
        remain_intestine = s["Intestine_Lumen"] - (mucus_trap + absorb_epith + fecal_loss)

        # --- Mucus layer dynamics ---
        absorb_from_mucus = s["Mucus_Layer"] * p["absorb_epith"]
        detach_from_mucus = s["Mucus_Layer"] * p["mucus_detach"]
        remain_mucus = s["Mucus_Layer"] - (absorb_from_mucus + detach_from_mucus)

        # --- Epithelium dynamics ---
        enter_blood = s["Epithelium"] * p["enter_blood"]
        detach_from_epith = s["Epithelium"] * p["epith_detach"]
        remain_epith = s["Epithelium"] - (enter_blood + detach_from_epith)

        # --- Blood to liver ---
        to_liver = enter_blood * p["to_liver"]
        return_from_liver = s["Liver"] * p["liver_detach"]
        remain_blood = s["Blood"] + enter_blood + return_from_liver - to_liver

        # --- Liver clearance ---
        hepatic_clear = s["Liver"] * p["hepatic_clear"]
        remain_liver = s["Liver"] + to_liver - (hepatic_clear + return_from_liver)

        # --- Assign new values ---
        new_state[cls]["Stomach_Lumen"] += remain_stomach
        new_state[cls]["Intestine_Lumen"] += remain_intestine + move_to_intestine + detach_from_mucus
        new_state[cls]["Mucus_Layer"] += remain_mucus + mucus_trap + detach_from_epith
        new_state[cls]["Epithelium"] += remain_epith + absorb_epith + absorb_from_mucus
        new_state[cls]["Blood"] += remain_blood
        new_state[cls]["Liver"] += remain_liver
        new_state[cls]["Fecal_Excretion"] += s["Fecal_Excretion"] + fecal_loss + hepatic_clear

        # --- Mass conservation correction ---
        total_prev = float(sum(s.values()))
        total_now = float(sum(new_state[cls].values()))
        diff = total_prev - total_now
        if abs(diff) > 1e-8:
            new_state[cls]["Stomach_Lumen"] += diff
            for k in new_state[cls]:
                if new_state[cls][k] < 0:
                    new_state[cls][k] = 0.0

    state = new_state

# ============================================================
# 6. Convert results to DataFrames
# ============================================================
times = np.arange(0, total_hours + dt_hours, dt_hours)
dfs = {comp: pd.DataFrame({cls: [h[cls][comp] for h in history] for cls in MP_CLASSES}, index=times)
       for comp in COMPARTMENTS}

# ============================================================
# 7. Plot results
# ============================================================
def plot_df(df, title, ylabel):
    plt.figure(figsize=(9, 6))
    for cls in MP_CLASSES:
        plt.plot(df.index, df[cls], label=f"{cls} ({SIZE_RANGES[cls]})")
    plt.title(title)
    plt.xlabel("Time (hours)")
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.show()

plot_df(dfs["Stomach_Lumen"], "Particles in Stomach", "Count")
plot_df(dfs["Intestine_Lumen"], "Particles in Intestine", "Count")
plot_df(dfs["Blood"], "Particles in Blood", "Count")
plot_df(dfs["Liver"], "Particles in Liver", "Count")
plot_df(dfs["Fecal_Excretion"], "Cumulative Excreted (Stool)", "Count")

# ============================================================
# 8. Final numeric summary
# ============================================================
print("\n--- FINAL NUMERIC SUMMARY ---")
for cls in MP_CLASSES:
    ini = INITIAL[cls]
    final = state[cls]
    total = sum(final.values())
    print(f"\n{cls} Particles (Initial {ini}, {SIZE_RANGES[cls]}):")
    for c in COMPARTMENTS:
        val = final[c]
        print(f"  {c:18s}: {val:8.1f} ({val/ini*100:.2f}%)")
    print(f"  Total conserved     : {total:.1f} / {ini} ({total/ini*100:.2f}%)")


In [ ]:
# Respiratory digital twin — compartmental microplastic fate model
# 0.5-hour timesteps, 48 hours total
# Size classes: Nano (<1 µm), Micro (1–100 µm), Macro (100 µm–1–4 mm)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------
# Helper: numerically-safe binomial
# -------------------------
def safe_binomial(n, p, rng):
    if n <= 0 or p <= 0:
        return 0
    if p >= 1.0:
        return int(n)
    # for reasonably large n use approximation to avoid overflow
    threshold = 1_000_000
    if n <= threshold:
        return int(rng.binomial(int(n), float(p)))
    else:
        return int(round(n * p))

# -------------------------
# Simulation configuration
# -------------------------
TOTAL_HOURS = 48
DT_HOURS = 0.5
STEPS = int(TOTAL_HOURS / DT_HOURS)
times = np.arange(0, TOTAL_HOURS + DT_HOURS, DT_HOURS)

rng = np.random.default_rng(2025)

# -------------------------
# Particle classes / sizes / initial loads
# -------------------------
MP_CLASSES = ["Nano", "Micro", "Macro"]
SIZE_RANGES = {
    "Nano": "<1 µm (true nanoplastics)",
    "Micro": "1–100 µm",
    "Macro": "100 µm – 1–4 mm"
}
INITIAL = {"Nano": 1000, "Micro": 700, "Macro": 300}

# -------------------------
# Compartments (keeps same set/format as your other twins)
# -------------------------
COMPS = [
    "Inhaled_Airways",   # nasal/oral cavity + trachea
    "Bronchi",           # larger conducting airways
    "Alveoli",           # gas-exchange region
    "Macrophage",        # phagocytosed fraction
    "Lymph_Node",        # lymphatic store / transit
    "Lung_Tissue",       # retained in parenchyma/interstitium
    "Passed_to_Gut",     # mucociliary clearance swallowed
    "Exhaled",           # expelled
    "Bloodstream"        # translocated across alveolar barrier
]

# -------------------------
# Transition & detachment rates (per 0.5 h step)
# Biological rationale (literature-consistent trends):
# - Nano: high alveolar penetration & translocation, higher detachment mobility.
# - Micro: moderate alveolar deposition, macrophage-dominant clearance.
# - Macro: mostly captured in airways and cleared to gut or exhaled.
# -------------------------
RATES = {
    "Nano": {
        "air_to_bronchi":       0.30,
        "bronchi_to_alveoli":   0.18,
        "bronchi_to_clear":     0.40,   # mucociliary → gut
        "exhaled_from_air":     0.05,
        "alveoli_to_macrophage":0.12,
        "alveoli_to_tissue":    0.10,
        "alveoli_to_blood":     0.03,
        "macrophage_to_alveoli":0.08,   # macrophage release / regurgitation
        "macrophage_to_lymph":  0.05,
        "lymph_to_gut":         0.30,
        "lymph_detach_to_macro": 0.02,   # lymph --> macrophage (small)
        "tissue_to_blood":      0.04,   # tissue detachment -> blood
        "blood_detach_to_alv":   0.005   # tiny backflow to alveoli/interstitium
    },
    "Micro": {
        "air_to_bronchi":       0.40,
        "bronchi_to_alveoli":   0.07,
        "bronchi_to_clear":     0.70,
        "exhaled_from_air":     0.06,
        "alveoli_to_macrophage":0.20,
        "alveoli_to_tissue":    0.12,
        "alveoli_to_blood":     0.005,
        "macrophage_to_alveoli":0.04,
        "macrophage_to_lymph":  0.05,
        "lymph_to_gut":         0.30,
        "lymph_detach_to_macro": 0.01,
        "tissue_to_blood":      0.015,
        "blood_detach_to_alv":   0.0005
    },
    "Macro": {
        "air_to_bronchi":       0.55,
        "bronchi_to_alveoli":   0.02,
        "bronchi_to_clear":     0.80,
        "exhaled_from_air":     0.10,
        "alveoli_to_macrophage":0.25,
        "alveoli_to_tissue":    0.18,
        "alveoli_to_blood":     0.0001,
        "macrophage_to_alveoli":0.02,
        "macrophage_to_lymph":  0.03,
        "lymph_to_gut":         0.30,
        "lymph_detach_to_macro": 0.005,
        "tissue_to_blood":      0.002,
        "blood_detach_to_alv":   0.00005
    }
}

# -------------------------
# Initialize state (integers)
# -------------------------
state = {cls: {c: 0 for c in COMPS} for cls in MP_CLASSES}
for cls in MP_CLASSES:
    state[cls]["Inhaled_Airways"] = int(INITIAL[cls])

history = []

# -------------------------
# Simulation loop (stochastic but mass-conservative)
# -------------------------
for step in range(STEPS + 1):
    # store snapshot (deep-ish copy)
    history.append({cls: {c: int(state[cls][c]) for c in COMPS} for cls in MP_CLASSES})
    if step == STEPS:
        break

    # prepare next state as copy of current (we will update)
    new_state = {cls: {c: int(state[cls][c]) for c in COMPS} for cls in MP_CLASSES}

    for cls in MP_CLASSES:
        r = RATES[cls]
        s = state[cls]

        # 1) Inhaled Airways -> Bronchi and Exhaled
        air = int(s["Inhaled_Airways"])
        to_bronchi = safe_binomial(air, r["air_to_bronchi"], rng)
        exhaled_from_air = safe_binomial(air - to_bronchi, r["exhaled_from_air"], rng)
        remain_air = air - to_bronchi - exhaled_from_air

        # 2) Bronchi -> Alveoli or cleared to gut (mucociliary)
        bronchi = int(s["Bronchi"]) + to_bronchi
        to_alveoli = safe_binomial(bronchi, r["bronchi_to_alveoli"], rng)
        to_clear_mucus = safe_binomial(bronchi - to_alveoli, r["bronchi_to_clear"], rng)
        remain_bronchi = bronchi - to_alveoli - to_clear_mucus

        # 3) Alveoli processes: to macrophage, tissue, blood
        alveoli = int(s["Alveoli"]) + to_alveoli
        to_mac = safe_binomial(alveoli, r["alveoli_to_macrophage"], rng)
        to_tissue = safe_binomial(alveoli - to_mac, r["alveoli_to_tissue"], rng)
        to_blood = safe_binomial(alveoli - to_mac - to_tissue, r["alveoli_to_blood"], rng)
        remain_alveoli = alveoli - (to_mac + to_tissue + to_blood)

        # 4) Macrophage dynamics: includes inflow from alveoli and small detach from lymph
        macroph = int(s["Macrophage"]) + to_mac
        # lymph_detach_into_macrophage (small)
        lymph_detach = safe_binomial(int(s["Lymph_Node"]), r["lymph_detach_to_macro"], rng)
        macroph += lymph_detach

        # macrophage -> lymph (drainage) and macrophage detach back to alveoli and retention
        to_lymph_from_mac = safe_binomial(macroph, r["macrophage_to_lymph"], rng)
        detach_from_mac = safe_binomial(macroph - to_lymph_from_mac, r["macrophage_to_alveoli"], rng)
        stay_mac = macroph - to_lymph_from_mac - detach_from_mac
        remain_macrophage = stay_mac  # what remains as macrophage store

        # 5) Lymph node dynamics: receives from macrophage, some goes to gut, some detach back
        lymph = int(s["Lymph_Node"]) + to_lymph_from_mac
        to_gut_from_lymph = safe_binomial(lymph, r["lymph_to_gut"], rng)
        lymph_detach_back = safe_binomial(lymph - to_gut_from_lymph, r["lymph_detach_to_macro"], rng)
        remain_lymph = lymph - to_gut_from_lymph - lymph_detach_back  # lymph_detach_back returns to macrophage above (we already used s["Lymph_Node"] in macroph step)

        # 6) Lung tissue dynamics: receives from alveoli and macrophage release, some detaches to blood
        tissue = int(s["Lung_Tissue"]) + to_tissue + detach_from_mac
        tissue_detach_to_blood = safe_binomial(tissue, r["tissue_to_blood"], rng)
        remain_tissue = tissue - tissue_detach_to_blood

        # 7) Blood dynamics: receives alveolar translocation + tissue detachment; small backflow to alveoli
        blood = int(s["Bloodstream"]) + to_blood + tissue_detach_to_blood
        blood_detach_to_alv = safe_binomial(blood, r["blood_detach_to_alv"], rng)
        blood_after_detach = blood - blood_detach_to_alv

        # 8) Passed_to_Gut gets the mucociliary cleared fraction
        passed_to_gut_add = to_clear_mucus + to_gut_from_lymph

        # 9) Exhaled gets exhaled fraction from air (we accumulated exhaled over time)
        exhaled_add = exhaled_from_air

        # --- update new_state for this class (mass conservative)
        new_state[cls]["Inhaled_Airways"] = remain_air
        new_state[cls]["Bronchi"] = remain_bronchi
        new_state[cls]["Alveoli"] = remain_alveoli + blood_detach_to_alv  # alveoli receive detach from blood
        new_state[cls]["Macrophage"] = remain_macrophage
        new_state[cls]["Lymph_Node"] = remain_lymph
        new_state[cls]["Lung_Tissue"] = remain_tissue
        new_state[cls]["Passed_to_Gut"] += passed_to_gut_add
        new_state[cls]["Exhaled"] += exhaled_add
        new_state[cls]["Bloodstream"] = blood_after_detach

        # --- mass conservation correction (tiny numerical fixes)
        total_now = sum(new_state[cls].values())
        diff = INITIAL[cls] - total_now
        if diff != 0:
            # put small correction into Airways (safe reservoir)
            new_state[cls]["Inhaled_Airways"] += diff
            if new_state[cls]["Inhaled_Airways"] < 0:
                # if negative, push remainder to Exhaled
                new_state[cls]["Exhaled"] += new_state[cls]["Inhaled_Airways"]
                new_state[cls]["Inhaled_Airways"] = 0

    # commit new_state
    state = new_state

# -------------------------
# Convert history -> DataFrames (one DataFrame per compartment, columns = classes)
# -------------------------
dfs = {}
for comp in COMPS:
    dfs[comp] = pd.DataFrame({cls: [history[i][cls][comp] for i in range(len(history))] for cls in MP_CLASSES},
                              index=times)

# -------------------------
# Plotting: keep same format as other twins
# -------------------------
for comp in COMPS:
    plt.figure(figsize=(9,4))
    for cls, color in zip(MP_CLASSES, ["blue","green","red"]):
        plt.plot(dfs[comp].index, dfs[comp][cls], label=f"{cls} ({SIZE_RANGES[cls]})", color=color)
    plt.title(comp.replace("_"," "))
    plt.xlabel("Time (hours)")
    plt.ylabel("Particles")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# -------------------------
# Final numeric summary (same format as your other programs)
# -------------------------
print("\n--- FINAL NUMERIC SUMMARY (respiratory) ---")
final_state = {cls: {c: int(state[cls][c]) for c in COMPS} for cls in MP_CLASSES}
for cls in MP_CLASSES:
    ini = INITIAL[cls]
    tot = sum(final_state[cls].values())
    print(f"\n{cls} particles (Initial {ini}, {SIZE_RANGES[cls]}):")
    for c in COMPS:
        val = final_state[cls][c]
        pct = (val/ini*100.0) if ini>0 else 0.0
        print(f"  {c:15s}: {val:8d} ({pct:6.2f}%)")
    print(f"  Total check        : {tot:8d} ({tot/ini*100.0:6.2f}%)")

# also show a neat dataframe
final_df = pd.DataFrame(final_state).T[COMPS]
display(final_df)


In [ ]:
# Excretory system (Kidney) Digital Twin with Detachment Rates
# 30-minute time steps, 48 hours total
# Nano, Micro, and Macro plastics
# Includes biologically realistic detachment probabilities

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------------
# Simulation configuration
# ----------------------------
DT_HOURS = 0.5           # 30 min per step
TOTAL_HOURS = 48
STEPS = int(TOTAL_HOURS / DT_HOURS)
TIMES = np.arange(0, TOTAL_HOURS + DT_HOURS, DT_HOURS)

# ----------------------------
# Particle classes and size ranges
# ----------------------------
MP_CLASSES = ["Nano", "Micro", "Macro"]
SIZE_RANGES = {
    "Nano": "<1 µm (true nanoplastics)",
    "Micro": "1–100 µm",
    "Macro": "100 µm – 1–4 mm"
}

# Initial particle counts entering blood
INITIAL = {"Nano": 1000, "Micro": 700, "Macro": 300}

# ----------------------------
# Compartments
# ----------------------------
COMPARTMENTS = [
    "Blood",
    "Kidney_Filter",
    "Reabsorbed_to_Blood",
    "Kidney_Tissue",
    "Urine_Tube",
    "Bladder",
    "Excreted"
]

# ----------------------------
# Rate parameters per 0.5-hour step
# Biologically plausible and literature-guided
# ----------------------------
RATES = {
    "Nano": {
        "blood_to_filter": 0.40,      # freely filtered
        "filter_to_reabsorb": 0.25,   # can be reabsorbed
        "filter_to_tissue": 0.02,     # some embed in tissue
        "filter_to_urine": 0.20,      # pass into urine
        "tissue_to_blood": 0.05,      # detach from tissue to blood
        "filter_to_blood_detach": 0.03, # detach from filter back to blood
        "urine_to_bladder": 0.80,
        "bladder_to_excrete": 0.80
    },
    "Micro": {
        "blood_to_filter": 0.30,
        "filter_to_reabsorb": 0.10,
        "filter_to_tissue": 0.04,
        "filter_to_urine": 0.25,
        "tissue_to_blood": 0.02,
        "filter_to_blood_detach": 0.015,
        "urine_to_bladder": 0.75,
        "bladder_to_excrete": 0.75
    },
    "Macro": {
        "blood_to_filter": 0.20,
        "filter_to_reabsorb": 0.02,
        "filter_to_tissue": 0.10,
        "filter_to_urine": 0.10,
        "tissue_to_blood": 0.005,
        "filter_to_blood_detach": 0.005,
        "urine_to_bladder": 0.60,
        "bladder_to_excrete": 0.60
    }
}

# ----------------------------
# Initialize states
# ----------------------------
state = {cls: {comp: 0.0 for comp in COMPARTMENTS} for cls in MP_CLASSES}
for cls in MP_CLASSES:
    state[cls]["Blood"] = float(INITIAL[cls])
history = []

# ----------------------------
# Simulation loop
# ----------------------------
for step in range(STEPS + 1):
    snapshot = {cls: dict(state[cls]) for cls in MP_CLASSES}
    history.append(snapshot)
    if step == STEPS:
        break

    new_state = {cls: {comp: 0.0 for comp in COMPARTMENTS} for cls in MP_CLASSES}

    for cls in MP_CLASSES:
        r = RATES[cls]
        s = state[cls]

        # 1. Blood → Filter
        to_filter = s["Blood"] * r["blood_to_filter"]
        remain_blood = s["Blood"] - to_filter
        new_state[cls]["Kidney_Filter"] += to_filter
        new_state[cls]["Blood"] += remain_blood

        # 2. Filter dynamics (to reabsorb, tissue, urine, detach)
        filt = s["Kidney_Filter"] + to_filter
        to_reabsorb = filt * r["filter_to_reabsorb"]
        to_tissue = filt * r["filter_to_tissue"]
        to_urine = filt * r["filter_to_urine"]
        to_detach_blood = filt * r["filter_to_blood_detach"]
        remain_filter = filt - (to_reabsorb + to_tissue + to_urine + to_detach_blood)

        new_state[cls]["Reabsorbed_to_Blood"] += to_reabsorb
        new_state[cls]["Kidney_Tissue"] += to_tissue
        new_state[cls]["Urine_Tube"] += to_urine
        new_state[cls]["Blood"] += to_detach_blood
        new_state[cls]["Kidney_Filter"] += remain_filter

        # 3. Tissue detachment → Blood
        tiss = s["Kidney_Tissue"]
        to_blood = tiss * r["tissue_to_blood"]
        remain_tiss = tiss - to_blood
        new_state[cls]["Kidney_Tissue"] += remain_tiss
        new_state[cls]["Blood"] += to_blood

        # 4. Urine → Bladder
        urine = s["Urine_Tube"] + to_urine
        to_bladder = urine * r["urine_to_bladder"]
        remain_urine = urine - to_bladder
        new_state[cls]["Urine_Tube"] += remain_urine
        new_state[cls]["Bladder"] += to_bladder

        # 5. Bladder → Excretion
        bladder = s["Bladder"] + to_bladder
        to_excrete = bladder * r["bladder_to_excrete"]
        remain_bladder = bladder - to_excrete
        new_state[cls]["Bladder"] += remain_bladder
        new_state[cls]["Excreted"] += to_excrete

        # 6. Add reabsorbed MPs to blood
        new_state[cls]["Blood"] += new_state[cls]["Reabsorbed_to_Blood"]

        # 7. Conservation correction
        total_now = sum(new_state[cls].values())
        diff = INITIAL[cls] - total_now
        if abs(diff) > 1e-6:
            new_state[cls]["Blood"] += diff

    state = new_state

# ----------------------------
# Convert results
# ----------------------------
dfs = {}
for cls in MP_CLASSES:
    df_rows = []
    for h in history:
        row = {comp: h[cls][comp] for comp in COMPARTMENTS}
        df_rows.append(row)
    dfs[cls] = pd.DataFrame(df_rows, index=TIMES)

# ----------------------------
# Plot results
# ----------------------------
for cls in MP_CLASSES:
    df = dfs[cls]
    plt.figure(figsize=(12,6))
    for comp in COMPARTMENTS:
        plt.plot(df.index, df[comp], label=comp)
    plt.title(f"{cls} particles ({SIZE_RANGES[cls]}) — Distribution in excretory system over {TOTAL_HOURS} h")
    plt.xlabel("Time (hours)")
    plt.ylabel("Particles")
    plt.legend(bbox_to_anchor=(1.02, 1.0), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# ----------------------------
# Conservation check
# ----------------------------
plt.figure(figsize=(10,5))
for cls, color in zip(MP_CLASSES, ["blue", "green", "red"]):
    total_ts = dfs[cls].sum(axis=1)
    plt.plot(dfs[cls].index, total_ts, label=f"Total {cls}", color=color)
plt.title("Total particle count check (mass conservation)")
plt.xlabel("Time (hours)")
plt.ylabel("Particles")
plt.legend()
plt.grid(True)
plt.show()

# ----------------------------
# Final numeric summary
# ----------------------------
print("\n--- FINAL NUMERIC SUMMARY ---")
for cls in MP_CLASSES:
    final = dfs[cls].iloc[-1]
    initial = INITIAL[cls]
    print(f"\n{cls} particles ({SIZE_RANGES[cls]}) (Initial {int(initial)}):")
    for comp in COMPARTMENTS:
        val = final[comp]
        pct = val / initial * 100
        print(f"  {comp:18s}: {val:8.1f} ({pct:6.2f}%)")
    print(f"  Total check          : {final.sum():8.1f} ({(final.sum()/initial*100):6.2f}%)")

print("\nExcretory system twin simulation complete with detachment modeling.")


In [ ]:
# Liver digital twin — compartmental model for microplastic kinetics
# 30-minute timesteps, 48 hours total
# Particle classes use biologically realistic size ranges and detachment rates
# (Fixed mass-conservation: prevents negative early values)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------------
# Simulation configuration
# ----------------------------
DT_HOURS = 0.5            # 30 min per step
TOTAL_HOURS = 48
STEPS = int(TOTAL_HOURS / DT_HOURS)
TIMES = np.arange(0, TOTAL_HOURS + DT_HOURS, DT_HOURS)

# ----------------------------
# Particle class definitions (range-based)
# ----------------------------
mp_types = {
    'Nano': {
        'size_range': '<1 µm (true nanoplastics)',
        'initial_count': 1000,
        'color': 'blue'
    },
    'Micro': {
        'size_range': '1–100 µm',
        'initial_count': 700,
        'color': 'green'
    },
    'Macro': {
        'size_range': '100 µm – 1–4 mm',
        'initial_count': 300,
        'color': 'red'
    }
}

MP_CLASSES = list(mp_types.keys())
INITIAL = {cls: mp_types[cls]['initial_count'] for cls in MP_CLASSES}

# ----------------------------
# Compartments
# ----------------------------
COMPARTMENTS = [
    "Portal_Vein",      # input blood from gut
    "Sinusoids",        # liver capillary network
    "Kupffer",          # macrophages
    "Hepatocytes",      # main parenchymal cells
    "Liver_Tissue",     # interstitial + retained
    "Bile",             # bile canaliculi
    "Bile_Excreted",    # excreted fraction
    "Hepatic_Vein"      # output back to circulation
]

# ----------------------------
# Rate parameters (per timestep = 0.5 h)
# ----------------------------
RATES = {
    "Nano": {
        "portal_to_sinus": 0.90,
        "sinus_to_kupffer": 0.03,
        "sinus_to_hepatocyte": 0.06,
        "sinus_passthrough_to_hepatic_vein": 0.80,
        "kupffer_to_tissue": 0.02,
        "kupffer_detach_to_hepatic_vein": 0.025,
        "hepatocyte_to_bile": 0.05,
        "hepatocyte_to_hepatic_vein": 0.03,
        "hepatocyte_to_tissue": 0.01,
        "hepatocyte_detach_to_hepatic_vein": 0.008,
        "tissue_detach_to_hepatic_vein": 0.002,
        "bile_to_bile_excreted": 0.80
    },
    "Micro": {
        "portal_to_sinus": 0.90,
        "sinus_to_kupffer": 0.07,
        "sinus_to_hepatocyte": 0.02,
        "sinus_passthrough_to_hepatic_vein": 0.81,
        "kupffer_to_tissue": 0.04,
        "kupffer_detach_to_hepatic_vein": 0.012,
        "hepatocyte_to_bile": 0.03,
        "hepatocyte_to_hepatic_vein": 0.01,
        "hepatocyte_to_tissue": 0.01,
        "hepatocyte_detach_to_hepatic_vein": 0.003,
        "tissue_detach_to_hepatic_vein": 0.001,
        "bile_to_bile_excreted": 0.70
    },
    "Macro": {
        "portal_to_sinus": 0.90,
        "sinus_to_kupffer": 0.18,
        "sinus_to_hepatocyte": 0.005,
        "sinus_passthrough_to_hepatic_vein": 0.715,
        "kupffer_to_tissue": 0.10,
        "kupffer_detach_to_hepatic_vein": 0.001,
        "hepatocyte_to_bile": 0.005,
        "hepatocyte_to_hepatic_vein": 0.002,
        "hepatocyte_to_tissue": 0.02,
        "hepatocyte_detach_to_hepatic_vein": 0.0003,
        "tissue_detach_to_hepatic_vein": 0.0001,
        "bile_to_bile_excreted": 0.60
    }
}

# ----------------------------
# Helper: remove excess safely (when total_now > initial)
# remove from compartments in preferred order until excess is cleared
# preferred removal order: Hepatic_Vein, Sinusoids, Portal_Vein, Kupffer, Hepatocytes, Liver_Tissue, Bile, Bile_Excreted
# ----------------------------
def remove_excess_from_state(ns, excess):
    """ns: dict of compartment->value; excess: positive number to remove"""
    order = ["Hepatic_Vein","Sinusoids","Portal_Vein","Kupffer","Hepatocytes","Liver_Tissue","Bile","Bile_Excreted"]
    rem = excess
    for comp in order:
        if rem <= 0:
            break
        available = ns.get(comp, 0.0)
        take = min(available, rem)
        ns[comp] = available - take
        rem -= take
    # if still rem>0 (shouldn't happen), subtract evenly (last-resort)
    if rem > 1e-12:
        # distribute small negative adjustments across all positive compartments proportionally
        positives = [c for c,v in ns.items() if v>0]
        if positives:
            total_pos = sum(ns[c] for c in positives)
            if total_pos>0:
                for c in positives:
                    frac = ns[c]/total_pos
                    ns[c] -= frac * rem
                    if ns[c] < 0:
                        ns[c] = 0.0
    return ns

# ----------------------------
# Initialize state
# ----------------------------
state = {cls: {comp: 0.0 for comp in COMPARTMENTS} for cls in MP_CLASSES}
for cls in MP_CLASSES:
    state[cls]["Portal_Vein"] = float(INITIAL[cls])

history = []

# ----------------------------
# Simulation loop
# ----------------------------
for step in range(STEPS + 1):
    snapshot = {cls: {comp: state[cls][comp] for comp in COMPARTMENTS} for cls in MP_CLASSES}
    history.append(snapshot)
    if step == STEPS:
        break

    new_state = {cls: {comp: 0.0 for comp in COMPARTMENTS} for cls in MP_CLASSES}

    for cls in MP_CLASSES:
        r = RATES[cls]
        s = state[cls]

        # Portal → Sinusoids
        portal = s["Portal_Vein"]
        to_sinusoids = r["portal_to_sinus"] * portal
        remain_portal = portal - to_sinusoids
        new_state[cls]["Portal_Vein"] += remain_portal
        new_state[cls]["Sinusoids"] += to_sinusoids

        # Sinusoids → Kupffer / Hepatocytes / Hepatic Vein
        sin = s["Sinusoids"] + to_sinusoids
        to_kupffer = r["sinus_to_kupffer"] * sin
        to_hepatocyte = r["sinus_to_hepatocyte"] * sin
        to_hepatic_vein_pass = r["sinus_passthrough_to_hepatic_vein"] * sin
        remain_sin = max(0.0, sin - (to_kupffer + to_hepatocyte + to_hepatic_vein_pass))
        new_state[cls]["Sinusoids"] += remain_sin
        new_state[cls]["Kupffer"] += to_kupffer
        new_state[cls]["Hepatocytes"] += to_hepatocyte
        new_state[cls]["Hepatic_Vein"] += to_hepatic_vein_pass

        # Kupffer cells
        kup = s["Kupffer"]
        kup_to_tissue = r["kupffer_to_tissue"] * kup
        kup_detach = r["kupffer_detach_to_hepatic_vein"] * kup
        kup_remain = max(0.0, kup - (kup_to_tissue + kup_detach))
        new_state[cls]["Kupffer"] += kup_remain
        new_state[cls]["Liver_Tissue"] += kup_to_tissue
        new_state[cls]["Hepatic_Vein"] += kup_detach

        # Hepatocytes
        hep = s["Hepatocytes"]
        hep_to_bile = r["hepatocyte_to_bile"] * hep
        hep_to_vein = r["hepatocyte_to_hepatic_vein"] * hep
        hep_to_tissue = r["hepatocyte_to_tissue"] * hep
        hep_detach = r["hepatocyte_detach_to_hepatic_vein"] * hep
        hep_remain = max(0.0, hep - (hep_to_bile + hep_to_vein + hep_to_tissue + hep_detach))
        new_state[cls]["Hepatocytes"] += hep_remain
        new_state[cls]["Bile"] += hep_to_bile
        new_state[cls]["Hepatic_Vein"] += hep_to_vein + hep_detach
        new_state[cls]["Liver_Tissue"] += hep_to_tissue

        # Liver tissue
        tissue = s["Liver_Tissue"]
        tissue_detach = r["tissue_detach_to_hepatic_vein"] * tissue
        tissue_remain = max(0.0, tissue - tissue_detach)
        new_state[cls]["Liver_Tissue"] += tissue_remain
        new_state[cls]["Hepatic_Vein"] += tissue_detach

        # Bile → Excreted
        bile = s["Bile"] + new_state[cls]["Bile"]
        bile_to_excreted = r["bile_to_bile_excreted"] * bile
        bile_remain = max(0.0, bile - bile_to_excreted)
        new_state[cls]["Bile"] = bile_remain
        new_state[cls]["Bile_Excreted"] += bile_to_excreted

        # Mass conservation correction: robust
        # If total_now < initial  -> add missing mass to Portal_Vein
        # If total_now > initial  -> remove excess from compartments in sensible order
        total_now = sum(new_state[cls].values())
        diff = float(INITIAL[cls]) - total_now

        if diff > 0:
            new_state[cls]["Portal_Vein"] += diff
        elif diff < 0:
            excess = -diff
            new_state[cls] = remove_excess_from_state(new_state[cls], excess)

        # Final clamp (safety)
        for comp in COMPARTMENTS:
            if new_state[cls][comp] < 0:
                new_state[cls][comp] = 0.0

    state = new_state

# ----------------------------
# Convert to DataFrames
# ----------------------------
dfs = {}
for cls in MP_CLASSES:
    df_rows = []
    for h in history:
        row = {comp: h[cls][comp] for comp in COMPARTMENTS}
        df_rows.append(row)
    dfs[cls] = pd.DataFrame(df_rows, index=TIMES)

# ----------------------------
# Plots (same style)
# ----------------------------
for cls in MP_CLASSES:
    df = dfs[cls]
    plt.figure(figsize=(12,6))
    for comp in COMPARTMENTS:
        plt.plot(df.index, df[comp], label=comp)
    plt.title(f"{cls} ({mp_types[cls]['size_range']}) — Liver distribution over {TOTAL_HOURS} h")
    plt.xlabel("Time (hours)")
    plt.ylabel("Particles")
    plt.legend(bbox_to_anchor=(1.02, 1.0), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plt.figure(figsize=(10,5))
for cls in MP_CLASSES:
    total_ts = dfs[cls].sum(axis=1)
    plt.plot(dfs[cls].index, total_ts, label=f"Total {cls}", color=mp_types[cls]['color'])
plt.title("Total particles (conservation check)")
plt.xlabel("Time (hours)")
plt.ylabel("Particles")
plt.legend()
plt.grid(True)
plt.show()

# ----------------------------
# Final Summary
# ----------------------------
print("\n--- FINAL NUMERIC SUMMARY ---")
for cls in MP_CLASSES:
    final = dfs[cls].iloc[-1]
    initial = float(INITIAL[cls])
    total = final.sum()
    print(f"\n{cls} particles ({mp_types[cls]['size_range']}) (Initial {int(initial)}):")
    for comp in COMPARTMENTS:
        val = final[comp]
        pct = (val / initial * 100.0) if initial > 0 else 0.0
        print(f"  {comp:15s}: {val:9.2f} ({pct:6.2f}%)")
    print(f"  Total check       : {total:9.2f} ({(total/initial*100.0):6.2f}%)")

print("\n✅ Liver twin simulation complete with robust conservation and detachment rates.")


In [ ]:
# Cardiovascular System Digital Twin (fixed: no negative values, conservation enforced)
# Simulates 48 hours (0.5-hour steps) of microplastic circulation
# Includes Nano, Micro, and Macro particles with biological rates (dictionary kept same)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --------------------------------
# 1. Simulation Configuration
# --------------------------------
DT_HOURS = 0.5       # timestep (30 minutes)
TOTAL_HOURS = 48
STEPS = int(TOTAL_HOURS / DT_HOURS)
TIMES = np.arange(0, TOTAL_HOURS + DT_HOURS, DT_HOURS)

# --------------------------------
# 2. Particle Classes and Sizes
# --------------------------------
MP_CLASSES = ["Nano", "Micro", "Macro"]

SIZE_RANGES = {
    "Nano": "<1 µm (true nanoplastics)",
    "Micro": "1–100 µm",
    "Macro": "100 µm – 1–4 mm"
}

# Initial counts entering systemic circulation
INITIAL = {"Nano": 1000, "Micro": 700, "Macro": 300}

# --------------------------------
# 3. Cardiovascular Compartments
# --------------------------------
COMPARTMENTS = [
    "Systemic_Arteries",
    "Cardiac_Capillaries",
    "Systemic_Capillaries_Other",
    "Venous_Return",
    "Heart_Chambers",
    "Lung_Capillaries",
    "Lung_Veins",
    "Excreted"   # kept for compatibility (unused here but present)
]

# --------------------------------
# 4. Biological Flow and Detachment Rates (kept as you had them)
# --------------------------------
RATES = {
    "Nano": {
        "artery_to_cardiac": 0.10,
        "artery_to_systemic": 0.45,
        "cardiac_to_venous": 0.80,
        "systemic_to_venous": 0.60,
        "venous_to_heart": 0.85,
        "heart_to_lungs": 0.85,
        "lungs_to_artery": 0.90,
        "cardiac_detach_to_blood": 0.05,  # detachment back to blood (adds to arteries)
        "systemic_detach_to_blood": 0.03,
        "lung_detach_to_blood": 0.02
    },
    "Micro": {
        "artery_to_cardiac": 0.08,
        "artery_to_systemic": 0.35,
        "cardiac_to_venous": 0.70,
        "systemic_to_venous": 0.50,
        "venous_to_heart": 0.80,
        "heart_to_lungs": 0.80,
        "lungs_to_artery": 0.85,
        "cardiac_detach_to_blood": 0.03,
        "systemic_detach_to_blood": 0.015,
        "lung_detach_to_blood": 0.01
    },
    "Macro": {
        "artery_to_cardiac": 0.05,
        "artery_to_systemic": 0.20,
        "cardiac_to_venous": 0.40,
        "systemic_to_venous": 0.30,
        "venous_to_heart": 0.70,
        "heart_to_lungs": 0.70,
        "lungs_to_artery": 0.75,
        "cardiac_detach_to_blood": 0.01,
        "systemic_detach_to_blood": 0.005,
        "lung_detach_to_blood": 0.002
    }
}

# --------------------------------
# 5. Initialize State
# --------------------------------
state = {cls: {comp: 0.0 for comp in COMPARTMENTS} for cls in MP_CLASSES}
for cls in MP_CLASSES:
    state[cls]["Systemic_Arteries"] = float(INITIAL[cls])

history = []

# --------------------------------
# 6. Simulation Loop (safe, non-negative flows)
# --------------------------------
for step in range(STEPS + 1):
    # snapshot of start-of-step state
    snapshot = {cls: dict(state[cls]) for cls in MP_CLASSES}
    history.append(snapshot)
    if step == STEPS:
        break

    # prepare zeroed new_state
    new_state = {cls: {comp: 0.0 for comp in COMPARTMENTS} for cls in MP_CLASSES}

    for cls in MP_CLASSES:
        r = RATES[cls]
        s = state[cls]  # start-of-step amounts

        # ----- compute flows based only on start-of-step amounts s (avoid mixing new inflows) -----

        # 1) Arterial distribution (all computed from s["Systemic_Arteries"])
        art_avail = s["Systemic_Arteries"]
        to_cardiac = r["artery_to_cardiac"] * art_avail
        to_systemic = r["artery_to_systemic"] * art_avail
        # safety clamp (shouldn't be necessary if rates sum <=1, but just in case)
        total_art_out = min(art_avail, to_cardiac + to_systemic)
        # proportionally scale if overflow
        if to_cardiac + to_systemic > 0:
            scale = total_art_out / (to_cardiac + to_systemic)
        else:
            scale = 0.0
        to_cardiac *= scale
        to_systemic *= scale
        remain_art = art_avail - (to_cardiac + to_systemic)

        # 2) Capillary detachment from stores (detachments computed from s)
        detach_cardiac = r["cardiac_detach_to_blood"] * s["Cardiac_Capillaries"]
        detach_systemic = r["systemic_detach_to_blood"] * s["Systemic_Capillaries_Other"]

        # 3) Capillary -> venous flows (from s)
        to_venous_cardiac = r["cardiac_to_venous"] * s["Cardiac_Capillaries"]
        to_venous_systemic = r["systemic_to_venous"] * s["Systemic_Capillaries_Other"]

        # 4) Venous -> heart (use start-of-step venous + venous inflow this step)
        venous_start = s["Venous_Return"]
        venous_inflow = to_venous_cardiac + to_venous_systemic
        venous_total_available = venous_start + venous_inflow
        to_heart = r["venous_to_heart"] * venous_total_available
        remain_venous = venous_total_available - to_heart

        # 5) Heart chambers -> lungs (use start-of-step heart + incoming)
        heart_start = s["Heart_Chambers"]
        heart_inflow = to_heart
        heart_total = heart_start + heart_inflow
        to_lungs = r["heart_to_lungs"] * heart_total
        remain_heart = heart_total - to_lungs

        # 6) Lungs flows: detachment and forward to arteries (use start-of-step lung + inflow)
        lung_start = s["Lung_Capillaries"]
        lung_inflow = to_lungs
        lung_total = lung_start + lung_inflow
        detach_lung = r["lung_detach_to_blood"] * lung_start  # detach computed from start-of-step store
        to_artery = r["lungs_to_artery"] * lung_total
        # ensure we don't send out more than available
        if to_artery + detach_lung > lung_total:
            scale2 = lung_total / (to_artery + detach_lung) if (to_artery + detach_lung) > 0 else 0.0
            to_artery *= scale2
            detach_lung *= scale2
            remain_lung = 0.0
        else:
            remain_lung = lung_total - (to_artery + detach_lung)

        # ----- now add flows into new_state (all are >= 0 by construction) -----
        new_state[cls]["Cardiac_Capillaries"] += to_cardiac
        new_state[cls]["Systemic_Capillaries_Other"] += to_systemic
        new_state[cls]["Systemic_Arteries"] += remain_art  # arteries keep remainder

        # add detachments back to arterial pool
        new_state[cls]["Systemic_Arteries"] += detach_cardiac + detach_systemic

        # venous inflow accumulation
        new_state[cls]["Venous_Return"] += remain_venous

        # heart chambers (remainder after pumping to lungs)
        new_state[cls]["Heart_Chambers"] += remain_heart

        # lungs and artery return
        new_state[cls]["Lung_Capillaries"] += remain_lung
        new_state[cls]["Systemic_Arteries"] += to_artery + detach_lung

        # hepatic/other "Excreted" left unused but preserved (kept zero unless you route there)
        # note: we do not subtract explicitly from capillary stores here, because we based flows on s (start-of-step).
        # But to reflect the outflow from capillary stores, ensure those stores also receive proper remainder:
        # - For cardiac capillaries: remainder = s["Cardiac_Capillaries"] - (to_venous_cardiac + detach_cardiac)
        cardiac_remain = s["Cardiac_Capillaries"] - (to_venous_cardiac + detach_cardiac)
        cardiac_remain = max(0.0, cardiac_remain)
        new_state[cls]["Cardiac_Capillaries"] += cardiac_remain

        # - For systemic capillaries: remainder = s["Systemic_Capillaries_Other"] - (to_venous_systemic + detach_systemic)
        systemic_remain = s["Systemic_Capillaries_Other"] - (to_venous_systemic + detach_systemic)
        systemic_remain = max(0.0, systemic_remain)
        new_state[cls]["Systemic_Capillaries_Other"] += systemic_remain

        # - For Venous_Return we already placed remain_venous; ensure we also account any venous_start that wasn't moved:
        # (venous_start was included in venous_total_available; remainder already included in remain_venous)

        # - For Heart_Chambers we added remain_heart; any heart_start not moved is accounted for.

        # - For Lung_Capillaries we added remain_lung; lung_start accounted.

        # ----- Conservation correction (tiny numerical fix) -----
        total_now = sum(new_state[cls].values())
        diff = INITIAL[cls] - total_now
        # apply tiny diff to a safe compartment (Systemic_Arteries)
        if abs(diff) > 1e-9:
            new_state[cls]["Systemic_Arteries"] += diff
            # ensure no negatives after correction
            for comp in COMPARTMENTS:
                if new_state[cls][comp] < 0:
                    # move negative amount to Excreted (fallback) and clamp
                    deficit = -new_state[cls][comp]
                    new_state[cls][comp] = 0.0
                    new_state[cls]["Excreted"] += deficit

    # commit new_state as next state
    state = new_state

# --------------------------------
# 7. Convert to DataFrames
# --------------------------------
dfs = {}
for cls in MP_CLASSES:
    df_rows = []
    for h in history:
        row = {comp: h[cls][comp] for comp in COMPARTMENTS}
        df_rows.append(row)
    dfs[cls] = pd.DataFrame(df_rows, index=TIMES)

# --------------------------------
# 8. Visualization (same output format as your other twins)
# --------------------------------
for cls in MP_CLASSES:
    df = dfs[cls]
    plt.figure(figsize=(12,6))
    for comp in COMPARTMENTS:
        plt.plot(df.index, df[comp], label=comp)
    plt.title(f"{cls} particles ({SIZE_RANGES[cls]}) — Cardiovascular System")
    plt.xlabel("Time (hours)")
    plt.ylabel("Particle count")
    plt.legend(bbox_to_anchor=(1.02, 1.0), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Conservation check
plt.figure(figsize=(10,5))
for cls, color in zip(MP_CLASSES, ["blue", "green", "red"]):
    total_ts = dfs[cls].sum(axis=1)
    plt.plot(dfs[cls].index, total_ts, label=f"Total {cls}", color=color)
plt.title("Total Particle Conservation — Cardiovascular System")
plt.xlabel("Time (hours)")
plt.ylabel("Total Particles")
plt.legend()
plt.grid(True)
plt.show()

# --------------------------------
# 9. Final Numeric Summary (same style as other twins)
# --------------------------------
print("\n--- FINAL NUMERIC SUMMARY ---")
for cls in MP_CLASSES:
    final = dfs[cls].iloc[-1]
    initial = INITIAL[cls]
    print(f"\n{cls} particles ({SIZE_RANGES[cls]}) (Initial {int(initial)}):")
    for comp in COMPARTMENTS:
        val = final[comp]
        pct = (val / initial * 100) if initial > 0 else 0.0
        print(f"  {comp:25s}: {val:8.1f} ({pct:6.2f}%)")
    print(f"  Total check               : {final.sum():8.1f} ({(final.sum()/initial*100):6.2f}%)")

print("\nCardiovascular system twin simulation complete (no negatives, conservation enforced).")


In [ ]:
# --- Skin (Dermal) Microplastic Twin Simulation (Particle-Conserving Version) ---
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# ----------------------------
# Simulation Function
# ----------------------------
def simulate_dermal_twin(
    n_nano=1000, n_micro=700, n_macro=300, time_steps=100
):
    transitions = {
        "Nano": {
            "size": "<1 µm (true nanoplastics)",
            "clothing_to_surface": 0.35,
            "surface_to_follicle": 0.30,
            "surface_to_skin": 0.10,
            "surface_to_gut": 0.02,
            "surface_to_removed": 0.03,
            "follicles_detach": 0.05,
            "skin_detach": 0.02,
            "skin_to_blood": 0.01,
        },
        "Micro": {
            "size": "1–100 µm",
            "clothing_to_surface": 0.25,
            "surface_to_follicle": 0.20,
            "surface_to_skin": 0.07,
            "surface_to_gut": 0.015,
            "surface_to_removed": 0.04,
            "follicles_detach": 0.06,
            "skin_detach": 0.025,
            "skin_to_blood": 0.002,
        },
        "Macro": {
            "size": "100 µm – 1–4 mm",
            "clothing_to_surface": 0.10,
            "surface_to_follicle": 0.05,
            "surface_to_skin": 0.01,
            "surface_to_gut": 0.005,
            "surface_to_removed": 0.10,
            "follicles_detach": 0.05,
            "skin_detach": 0.03,
            "skin_to_blood": 0.0,
        },
    }

    # Track all compartments
    results = {
        ptype: {
            "Clothing": [], "Surface": [], "Follicles": [],
            "Skin_combined": [], "Passed_to_Gut": [],
            "Blood": [], "Removed_from_System": []
        } for ptype in transitions
    }

    initial_counts = {"Nano": n_nano, "Micro": n_micro, "Macro": n_macro}

    for ptype, pdata in transitions.items():
        clothing = initial_counts[ptype]
        surface = 0
        follicles = skin = gut = blood = removed = 0

        for _ in range(time_steps):
            # 1️⃣ Transfer from clothing to surface
            to_surface = clothing * pdata["clothing_to_surface"]
            clothing -= to_surface
            surface += to_surface

            # 2️⃣ Surface transfers
            to_follicle = surface * pdata["surface_to_follicle"]
            to_skin = surface * pdata["surface_to_skin"]
            to_gut = surface * pdata["surface_to_gut"]
            to_removed = surface * pdata["surface_to_removed"]

            # 3️⃣ Internal detachment and transfer
            detach_from_follicles = follicles * pdata["follicles_detach"]
            detach_from_skin = skin * pdata["skin_detach"]
            to_blood = skin * pdata["skin_to_blood"]

            # 4️⃣ Update compartments
            follicles += to_follicle - detach_from_follicles
            skin += to_skin - detach_from_skin - to_blood
            gut += to_gut
            blood += to_blood
            removed += to_removed + detach_from_follicles + detach_from_skin
            surface -= (to_follicle + to_skin + to_gut + to_removed)

            # 5️⃣ Numerical stability
            clothing = max(clothing, 0)
            surface = max(surface, 0)
            follicles = max(follicles, 0)
            skin = max(skin, 0)
            gut = max(gut, 0)
            blood = max(blood, 0)
            removed = max(removed, 0)

            # 6️⃣ Conservation correction
            total = clothing + surface + follicles + skin + gut + blood + removed
            correction = initial_counts[ptype] - total
            if abs(correction) > 1e-6:  # tiny rounding fix
                surface += correction

            # Store
            results[ptype]["Clothing"].append(clothing)
            results[ptype]["Surface"].append(surface)
            results[ptype]["Follicles"].append(follicles)
            results[ptype]["Skin_combined"].append(skin)
            results[ptype]["Passed_to_Gut"].append(gut)
            results[ptype]["Blood"].append(blood)
            results[ptype]["Removed_from_System"].append(removed)

    return results, transitions, initial_counts


# ----------------------------
# Display Results (Graphs first, then text)
# ----------------------------
def display_results(results, transitions, initial_counts):
    print("\n--- Dermal Twin Simulation (Conserved) ---\n")

    # 1️⃣ Plot all graphs first
    for ptype, values in results.items():
        plt.figure(figsize=(8, 5))
        for key, val in values.items():
            plt.plot(range(len(val)), val, label=key)
        plt.title(f"{ptype} Microplastics Over Time")
        plt.xlabel("Time Steps")
        plt.ylabel("Number of Particles")
        plt.legend()
        plt.grid(True)
        plt.show()

    # 2️⃣ Then written summaries
    for ptype, values in results.items():
        pdata = transitions[ptype]
        total_init = initial_counts[ptype]
        final_vals = {k: v[-1] for k, v in values.items()}
        total_check = sum(final_vals.values())

        print(f"{ptype} Particles (Initial {total_init}, {pdata['size']}):")
        for k, v in final_vals.items():
            print(f"  {k:<20}: {v:8.1f} ({(v/total_init)*100:6.2f}%)")
        print(f"  Total check         : {total_check:8.1f} (100.00%)\n")


# ----------------------------
# Run Model
# ----------------------------
results, transitions, initial_counts = simulate_dermal_twin()
display_results(results, transitions, initial_counts)


In [ ]:
# Lymphatic system digital twin (Colab / Jupyter ready)
# 30-minute timesteps, 48 hours total
# Three particle classes: Nano (<1um), Micro (1-100um), Macro (100um-1-4mm)
# Outputs: time-series plots and final numeric summary dictionary for each type
# Mass-conserving; detachment/clearance included where biologically plausible.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------------
# Simulation config
# ----------------------------
DT_HOURS = 0.5
TOTAL_HOURS = 48
STEPS = int(TOTAL_HOURS / DT_HOURS)
TIMES = np.arange(0, TOTAL_HOURS + DT_HOURS, DT_HOURS)  # index length STEPS+1

# ----------------------------
# Particle classes + sizes + initial counts
# ----------------------------
MP_CLASSES = ["Nano", "Micro", "Macro"]
SIZE_RANGES = {
    "Nano": "<1 µm (true nanoplastics)",
    "Micro": "1–100 µm",
    "Macro": "100 µm – 1–4 mm"
}
INITIAL = {"Nano": 1000.0, "Micro": 700.0, "Macro": 300.0}
COLORS = {"Nano": "blue", "Micro": "green", "Macro": "red"}

# ----------------------------
# Compartments (lymph-focused)
# ----------------------------
COMPARTMENTS = [
    "Interstitium",       # tissue interstitial fluid (source)
    "Initial_Lymphatics", # entry (lymphatic capillaries)
    "Lymph_Node",         # draining lymph node (trapping/retention)
    "Thoracic_Duct",      # collecting duct / conduit to venous return
    "Blood",              # systemic circulation (receives lymph)
    "Removed_from_System" # effective removal (macrophage digestion / sequestration)
]

# ----------------------------
# Rate parameters (per DT = 0.5 h)
# Biologically conservative, literature-informed choices:
# - uptake_to_lymph: fraction of Interstitium -> Initial_Lymphatics per 0.5 h
# - lymph_to_node_pass: fraction of Initial_Lymphatics that reaches node per step
# - node_trap: fraction of incoming lymph that is trapped/retained in node per step
# - node_detach: fraction of node store that can detach and re-enter flowing lymph per step
# - node_to_removed: fraction of node store that becomes irreversibly removed per step
# - thoracic_pass: fraction of Thoracic_Duct -> Blood per step (drainage)
# - small reverse flows (detachments) included: node_detach, initial_lymph_detach->interstitium (very small)
#
# Rationale summary:
# - Nanos: highest uptake into lymph, moderate node retention, some detachment (nano mobility)
# - Micros: moderate/low lymph uptake, higher node trapping fraction when they arrive
# - Macros: very low lymph uptake (mostly stay in interstitium), if they reach node they tend to be trapped
# ----------------------------
RATES = {
    "Nano": {
        "uptake_to_lymph": 0.30,      # 30% of interstitial nanos enter lymph per 0.5 h
        "lymph_flow_to_node": 0.90,   # fraction of lymph reaching node per step (flow)
        "node_trap": 0.25,            # node traps 25% of arriving per step
        "node_detach": 0.05,          # 5% of node store can detach back into lymph per step
        "node_to_removed": 0.02,      # 2% of node store becomes 'removed' per step (macrophage digestion/sequestration)
        "node_to_thoracic_pass": 0.70,# fraction of arriving lymph that passes through (complement of trap)
        "thoracic_to_blood": 0.95,    # thoracic duct empties to blood quickly (~per step)
        "initial_lymph_detach": 0.01  # tiny fraction re-diffuses back to interstitium per step
    },
    "Micro": {
        "uptake_to_lymph": 0.08,      # 8% uptake per 0.5h (much lower than nano)
        "lymph_flow_to_node": 0.92,
        "node_trap": 0.40,            # nodes trap a larger fraction of micros that arrive
        "node_detach": 0.01,          # low detachment
        "node_to_removed": 0.03,
        "node_to_thoracic_pass": 0.59,
        "thoracic_to_blood": 0.95,
        "initial_lymph_detach": 0.005
    },
    "Macro": {
        "uptake_to_lymph": 0.005,     # very small chance macro enters lymphatics
        "lymph_flow_to_node": 0.95,
        "node_trap": 0.70,            # if macro reaches node, it's largely trapped
        "node_detach": 0.001,
        "node_to_removed": 0.04,
        "node_to_thoracic_pass": 0.29,
        "thoracic_to_blood": 0.95,
        "initial_lymph_detach": 0.001
    }
}

# ----------------------------
# Initialize state (per-class)
# ----------------------------
state = {cls: {c: 0.0 for c in COMPARTMENTS} for cls in MP_CLASSES}
# For this twin we assume particles are *already present in Interstitium* at t=0.
for cls in MP_CLASSES:
    state[cls]["Interstitium"] = float(INITIAL[cls])

history = []

# ----------------------------
# Simulation loop (deterministic fraction flows with conservation)
# ----------------------------
for step in range(STEPS + 1):
    # snapshot current
    history.append({cls: {c: state[cls][c] for c in COMPARTMENTS} for cls in MP_CLASSES})
    if step == STEPS:
        break

    # prepare new state
    new_state = {cls: {c: 0.0 for c in COMPARTMENTS} for cls in MP_CLASSES}

    for cls in MP_CLASSES:
        r = RATES[cls]
        s = state[cls]

        # 1) Interstitium -> Initial_Lymphatics (size-dependent uptake)
        inter = s["Interstitium"]
        to_lymph = r["uptake_to_lymph"] * inter
        inter_remain = inter - to_lymph

        # small re-diffusion from initial lymph back to interstitium (detach)
        # we model this based on the previous existing initial lymphatics store
        init_lymph_current = s["Initial_Lymphatics"]
        init_to_inter = r["initial_lymph_detach"] * init_lymph_current

        # 2) Flow from Initial_Lymphatics to Lymph Node (carried by lymph flow)
        inflow_to_node = (s["Initial_Lymphatics"] + to_lymph - init_to_inter) * r["lymph_flow_to_node"]

        # remaining left in initial lymphatics
        init_remain = s["Initial_Lymphatics"] + to_lymph - init_to_inter - inflow_to_node
        init_remain = max(init_remain, 0.0)

        # 3) Node handling: fraction of inflow trapped, fraction passes onward
        node_trapped_this_step = inflow_to_node * r["node_trap"]
        node_pass_through = inflow_to_node * r["node_to_thoracic_pass"]
        # node store from previous step plus newly trapped
        node_current_store = s["Lymph_Node"]
        # Detach from node back into flowing lymph (releases)
        node_detach_back = r["node_detach"] * node_current_store
        # irreversible removal (e.g., macrophage digestion or permanent sequestration)
        node_to_removed = r["node_to_removed"] * node_current_store
        # node remaining after detach + removal
        node_remain = node_current_store - (node_detach_back + node_to_removed)
        node_remain = max(node_remain, 0.0)

        # The node releases the detached portion back into the lumen (contribute to thoracic flow)
        node_release_to_flow = node_detach_back

        # 4) Thoracic duct: receives the node_pass_through + node_release_to_flow
        thoracic_in = s["Thoracic_Duct"] + node_pass_through + node_release_to_flow
        # fraction of thoracic duct that empties into blood per step
        to_blood = r["thoracic_to_blood"] * thoracic_in
        thoracic_remain = thoracic_in - to_blood
        thoracic_remain = max(thoracic_remain, 0.0)

        # 5) Update compartments (conserve mass)
        new_state[cls]["Interstitium"] += inter_remain + init_to_inter  # return from initial lymph detach added back
        new_state[cls]["Initial_Lymphatics"] += init_remain
        new_state[cls]["Lymph_Node"] += node_remain + node_trapped_this_step  # store old remain + newly trapped
        new_state[cls]["Thoracic_Duct"] += thoracic_remain
        new_state[cls]["Blood"] += s["Blood"] + to_blood  # accumulate previously present blood + incoming
        new_state[cls]["Removed_from_System"] += s["Removed_from_System"] + node_to_removed

        # 6) Small numeric conservation correction (avoid floating drift)
        total_prev = sum([s[c] for c in COMPARTMENTS])
        total_now = sum([new_state[cls][c] for c in COMPARTMENTS])
        diff = float(INITIAL[cls]) - total_now
        if abs(diff) > 1e-9:
            # safely put tiny diff back into Interstitium
            new_state[cls]["Interstitium"] += diff
            if new_state[cls]["Interstitium"] < 0:
                rem = -new_state[cls]["Interstitium"]
                new_state[cls]["Interstitium"] = 0.0
                new_state[cls]["Removed_from_System"] += rem

    # commit
    state = new_state

# ----------------------------
# Build time-series DataFrames per compartment (for plotting)
# ----------------------------
dfs = {}
for cls in MP_CLASSES:
    rows = []
    for h in history:
        rows.append({comp: h[cls][comp] for comp in COMPARTMENTS})
    dfs[cls] = pd.DataFrame(rows, index=TIMES)

# ----------------------------
# Plots: compartments (same style as your other twins)
# ----------------------------
for comp in COMPARTMENTS:
    plt.figure(figsize=(8,4))
    for cls in MP_CLASSES:
        plt.plot(dfs[cls].index, dfs[cls][comp], label=f"{cls} ({SIZE_RANGES[cls]})", color=COLORS[cls])
    plt.title(f"{comp.replace('_',' ')} over time")
    plt.xlabel("Time (hours)")
    plt.ylabel("Particles")
    plt.legend()
    plt.grid(True)
    plt.show()

# ----------------------------
# Final numeric summary and result dictionary (same format style)
# ----------------------------
results = {}
print("\n--- FINAL NUMERIC SUMMARY ---")
for cls in MP_CLASSES:
    final = dfs[cls].iloc[-1]
    initial = float(INITIAL[cls])
    inter = final["Interstitium"]
    init_lymph = final["Initial_Lymphatics"]
    node = final["Lymph_Node"]
    thor = final["Thoracic_Duct"]
    blood = final["Blood"]
    removed = final["Removed_from_System"]
    total = final.sum()
    # same concise dictionary style you asked for
    results[cls] = {
        "size": SIZE_RANGES[cls],
        "Interstitium": inter,
        "Initial_Lymphatics": init_lymph,
        "Lymph_Node": node,
        "Thoracic_Duct": thor,
        "Blood": blood,
        "Removed_from_System": removed
    }

    print(f"\n{cls} Particles (Initial {int(initial)}, {SIZE_RANGES[cls]}):")
    for key in ["Interstitium","Initial_Lymphatics","Lymph_Node","Thoracic_Duct","Blood","Removed_from_System"]:
        val = results[cls][key]
        pct = (val/initial*100.0) if initial>0 else 0.0
        print(f"  {key:18s}: {val:8.1f} ({pct:6.2f}%)")
    print(f"  Total conserved     : {total:8.1f} / {initial:.0f} ({total/initial*100:.2f}%)")

# results dict is available programmatically for downstream linking
results_df = pd.DataFrame.from_dict({k: results[k] for k in results}, orient='index')
display(results_df)


In [ ]:
# Male Reproductive System Digital Twin
# Deterministic compartment model for Nano, Micro, Macro plastics
# 30-min timesteps, 48 hours total
# Mass conserved; biological assumptions included

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------------
# Simulation setup
# ----------------------------
DT_HOURS = 0.5
TOTAL_HOURS = 48
TIMES = np.arange(0, TOTAL_HOURS + DT_HOURS, DT_HOURS)

MP_CLASSES = ["Nano", "Micro", "Macro"]
SIZE_RANGES = {
    "Nano": "<1 µm (true nanoplastics)",
    "Micro": "1–100 µm",
    "Macro": "100 µm – 1–4 mm"
}
INITIAL = {"Nano": 1000.0, "Micro": 700.0, "Macro": 300.0}

# ----------------------------
# Male system compartments & rates
# ----------------------------
COMPARTMENTS = [
    "Blood",
    "Testes_Interstitial",
    "Epididymis",
    "Seminal_Fluid",
    "Lymph_Node",
    "Accumulated_Tissue",
    "Excreted"
]

RATES = {
    "Nano": {
        "blood_to_testes": 0.06,
        "blood_to_lymph": 0.02,
        "testes_to_epidid": 0.04,
        "testes_to_accum": 0.01,
        "testes_detach_to_blood": 0.01,
        "epidid_to_seminal": 0.04,
        "epidid_detach_to_blood": 0.005,
        "lymph_to_blood": 0.02,
        "seminal_to_excreted": 0.15,
        "accum_detach_to_blood": 0.001
    },
    "Micro": {
        "blood_to_testes": 0.02,
        "blood_to_lymph": 0.03,
        "testes_to_epidid": 0.03,
        "testes_to_accum": 0.02,
        "testes_detach_to_blood": 0.005,
        "epidid_to_seminal": 0.02,
        "epidid_detach_to_blood": 0.002,
        "lymph_to_blood": 0.02,
        "seminal_to_excreted": 0.12,
        "accum_detach_to_blood": 0.0008
    },
    "Macro": {
        "blood_to_testes": 0.005,
        "blood_to_lymph": 0.05,
        "testes_to_epidid": 0.01,
        "testes_to_accum": 0.05,
        "testes_detach_to_blood": 0.0005,
        "epidid_to_seminal": 0.005,
        "epidid_detach_to_blood": 0.0002,
        "lymph_to_blood": 0.03,
        "seminal_to_excreted": 0.10,
        "accum_detach_to_blood": 0.0002
    }
}

# ----------------------------
# Simulation function
# ----------------------------
def run_twin(mp_classes, compartments, rates, initial_counts, times):
    history = {cls: [] for cls in mp_classes}
    state = {cls: {c: 0.0 for c in compartments} for cls in mp_classes}
    for cls in mp_classes:
        state[cls]["Blood"] = float(initial_counts[cls])

    for step in range(len(times)):
        for cls in mp_classes:
            history[cls].append(state[cls].copy())

        if step == len(times) - 1:
            break

        new_state = {cls: {c: 0.0 for c in compartments} for cls in mp_classes}

        for cls in mp_classes:
            r = rates[cls]
            s = state[cls]
            B, T, E, S, L, A, X = compartments

            blood = s[B]
            to_testes = r["blood_to_testes"] * blood
            to_lymph = r["blood_to_lymph"] * blood
            remain_blood = blood - (to_testes + to_lymph)
            new_state[cls][B] += remain_blood

            lymph = s[L] + to_lymph
            lymph_to_blood = r["lymph_to_blood"] * lymph
            lymph_remain = lymph - lymph_to_blood
            new_state[cls][L] += lymph_remain
            new_state[cls][B] += lymph_to_blood

            testes = s[T] + to_testes
            t_to_epid = r["testes_to_epidid"] * testes
            t_to_accum = r["testes_to_accum"] * testes
            t_detach = r["testes_detach_to_blood"] * testes
            testes_remain = max(0.0, testes - (t_to_epid + t_to_accum + t_detach))
            new_state[cls][T] += testes_remain
            new_state[cls][E] += t_to_epid
            new_state[cls][A] += t_to_accum
            new_state[cls][B] += t_detach

            epid = s[E] + new_state[cls][E]
            e_to_seminal = r["epidid_to_seminal"] * epid
            e_detach = r["epidid_detach_to_blood"] * epid
            epid_remain = max(0.0, epid - (e_to_seminal + e_detach))
            new_state[cls][E] = epid_remain
            new_state[cls][S] += e_to_seminal
            new_state[cls][B] += e_detach

            sem = s[S] + new_state[cls][S]
            sem_excrete = r["seminal_to_excreted"] * sem
            sem_remain = max(0.0, sem - sem_excrete)
            new_state[cls][S] = sem_remain
            new_state[cls][X] += sem_excrete

            accum = s[A] + new_state[cls][A]
            accum_to_blood = r["accum_detach_to_blood"] * accum
            accum_remain = max(0.0, accum - accum_to_blood)
            new_state[cls][A] = accum_remain
            new_state[cls][B] += accum_to_blood

            total_now = sum(new_state[cls].values())
            diff = initial_counts[cls] - total_now
            new_state[cls][B] += diff

        state = new_state

    dfs = {cls: pd.DataFrame(history[cls], index=times) for cls in mp_classes}
    return dfs

# ----------------------------
# Run and plot
# ----------------------------
dfs = run_twin(MP_CLASSES, COMPARTMENTS, RATES, INITIAL, TIMES)

for comp in COMPARTMENTS:
    plt.figure(figsize=(8,4))
    for cls, color in zip(MP_CLASSES, ["blue", "green", "red"]):
        plt.plot(dfs[cls].index, dfs[cls][comp], label=f"{cls} ({SIZE_RANGES[cls]})", color=color)
    plt.title(f"Male Reproductive System — {comp.replace('_',' ')} over time")
    plt.xlabel("Time (hours)")
    plt.ylabel("Particles")
    plt.legend(); plt.grid(True); plt.show()

print("\n--- FINAL NUMERIC SUMMARY (Male) ---")
for cls in MP_CLASSES:
    df = dfs[cls]; final = df.iloc[-1]; ini = INITIAL[cls]
    print(f"\n{cls} particles (Initial {int(ini)}, {SIZE_RANGES[cls]}):")
    for comp in COMPARTMENTS:
        val = final[comp]; pct = (val/ini*100)
        print(f"  {comp:22s}: {val:8.2f} ({pct:6.2f}%)")
    print(f"  Total conserved     : {final.sum():8.2f} / {ini:.0f} ({final.sum()/ini*100:6.2f}%)")

print("\nMale reproductive twin complete.")


In [ ]:
# Female Reproductive System Digital Twin
# Deterministic compartment model for Nano, Micro, Macro plastics
# 30-min timesteps, 48 hours total
# Mass conserved; biological assumptions included

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------------
# Simulation setup
# ----------------------------
DT_HOURS = 0.5
TOTAL_HOURS = 48
TIMES = np.arange(0, TOTAL_HOURS + DT_HOURS, DT_HOURS)

MP_CLASSES = ["Nano", "Micro", "Macro"]
SIZE_RANGES = {
    "Nano": "<1 µm (true nanoplastics)",
    "Micro": "1–100 µm",
    "Macro": "100 µm – 1–4 mm"
}
INITIAL = {"Nano": 1000.0, "Micro": 700.0, "Macro": 300.0}

# ----------------------------
# Female system compartments & rates
# ----------------------------
COMPARTMENTS = [
    "Blood",
    "Ovarian_Interstitial",
    "Follicular_Fluid",
    "Uterine_Tissue",
    "Vaginal_Secretions",
    "Lymph_Node",
    "Accumulated_Tissue",
    "Excreted"
]

RATES = {
    "Nano": {
        "blood_to_ovary": 0.05,
        "blood_to_lymph": 0.02,
        "ovary_to_follicle": 0.04,
        "ovary_to_uterus": 0.02,
        "ovary_to_accum": 0.01,
        "ovary_detach_to_blood": 0.01,
        "follicle_to_blood": 0.01,
        "uterus_to_secretions": 0.06,
        "lymph_to_blood": 0.02,
        "accum_detach_to_blood": 0.001
    },
    "Micro": {
        "blood_to_ovary": 0.02,
        "blood_to_lymph": 0.03,
        "ovary_to_follicle": 0.02,
        "ovary_to_uterus": 0.03,
        "ovary_to_accum": 0.02,
        "ovary_detach_to_blood": 0.005,
        "follicle_to_blood": 0.002,
        "uterus_to_secretions": 0.04,
        "lymph_to_blood": 0.02,
        "accum_detach_to_blood": 0.0008
    },
    "Macro": {
        "blood_to_ovary": 0.005,
        "blood_to_lymph": 0.05,
        "ovary_to_follicle": 0.005,
        "ovary_to_uterus": 0.01,
        "ovary_to_accum": 0.06,
        "ovary_detach_to_blood": 0.0005,
        "follicle_to_blood": 0.0005,
        "uterus_to_secretions": 0.02,
        "lymph_to_blood": 0.03,
        "accum_detach_to_blood": 0.0002
    }
}

# ----------------------------
# Simulation function
# ----------------------------
def run_twin(mp_classes, compartments, rates, initial_counts, times):
    history = {cls: [] for cls in mp_classes}
    state = {cls: {c: 0.0 for c in compartments} for cls in mp_classes}
    for cls in mp_classes:
        state[cls]["Blood"] = float(initial_counts[cls])

    for step in range(len(times)):
        for cls in mp_classes:
            history[cls].append(state[cls].copy())
        if step == len(times) - 1:
            break

        new_state = {cls: {c: 0.0 for c in compartments} for cls in mp_classes}
        for cls in mp_classes:
            r = rates[cls]; s = state[cls]
            B,O,F,U,V,L,A,X = compartments

            blood = s[B]
            to_ovary = r["blood_to_ovary"] * blood
            to_lymph = r["blood_to_lymph"] * blood
            remain_blood = blood - (to_ovary + to_lymph)
            new_state[cls][B] += remain_blood

            lymph = s[L] + to_lymph
            lymph_to_blood = r["lymph_to_blood"] * lymph
            new_state[cls][L] += lymph - lymph_to_blood
            new_state[cls][B] += lymph_to_blood

            ov = s[O] + to_ovary
            o_to_fol = r["ovary_to_follicle"] * ov
            o_to_uter = r["ovary_to_uterus"] * ov
            o_to_accum = r["ovary_to_accum"] * ov
            o_detach = r["ovary_detach_to_blood"] * ov
            new_state[cls][O] += ov - (o_to_fol + o_to_uter + o_to_accum + o_detach)
            new_state[cls][F] += o_to_fol
            new_state[cls][U] += o_to_uter
            new_state[cls][A] += o_to_accum
            new_state[cls][B] += o_detach

            fol = s[F] + new_state[cls][F]
            f_to_blood = r["follicle_to_blood"] * fol
            new_state[cls][F] = fol - f_to_blood
            new_state[cls][B] += f_to_blood

            uter = s[U] + new_state[cls][U]
            u_to_sec = r["uterus_to_secretions"] * uter
            new_state[cls][U] = uter - u_to_sec
            new_state[cls][V] += u_to_sec

            vag = s[V] + new_state[cls][V]
            sec_excrete = 0.05 * vag
            new_state[cls][V] = vag - sec_excrete
            new_state[cls][X] += sec_excrete

            accum = s[A] + new_state[cls][A]
            a_to_blood = r["accum_detach_to_blood"] * accum
            new_state[cls][A] = accum - a_to_blood
            new_state[cls][B] += a_to_blood

            diff = initial_counts[cls] - sum(new_state[cls].values())
            new_state[cls][B] += diff

        state = new_state

    dfs = {cls: pd.DataFrame(history[cls], index=times) for cls in mp_classes}
    return dfs

# ----------------------------
# Run + plot + summary
# ----------------------------
dfs = run_twin(MP_CLASSES, COMPARTMENTS, RATES, INITIAL, TIMES)

for comp in COMPARTMENTS:
    plt.figure(figsize=(8,4))
    for cls, color in zip(MP_CLASSES, ["blue","green","red"]):
        plt.plot(dfs[cls].index, dfs[cls][comp], label=f"{cls} ({SIZE_RANGES[cls]})", color=color)
    plt.title(f"Female Reproductive System — {comp.replace('_',' ')} over time")
    plt.xlabel("Time (hours)"); plt.ylabel("Particles")
    plt.legend(); plt.grid(True); plt.show()

print("\n--- FINAL NUMERIC SUMMARY (Female) ---")
for cls in MP_CLASSES:
    df = dfs[cls]; final = df.iloc[-1]; ini = INITIAL[cls]
    print(f"\n{cls} particles (Initial {int(ini)}, {SIZE_RANGES[cls]}):")
    for comp in COMPARTMENTS:
        val = final[comp]; pct = (val/ini*100)
        print(f"  {comp:22s}: {val:8.2f} ({pct:6.2f}%)")
    print(f"  Total conserved     : {final.sum():8.2f} / {ini:.0f} ({final.sum()/ini*100:6.2f}%)")

print("\nFemale reproductive twin complete.")


In [ ]:
# Nervous system digital twin — Colab-ready
# 30-minute timesteps, 48 hours (deterministic, mass-conserving)
# Particle classes: Nano (<1 µm), Micro (1–100 µm), Macro (100 µm–1–4 mm)
# Outputs: plots (one per compartment across particle types) then numeric summary per particle class.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- Simulation configuration ----------
DT_HOURS = 0.5               # 30 min timestep
TOTAL_HOURS = 48
STEPS = int(TOTAL_HOURS / DT_HOURS)
TIMES = np.arange(0, TOTAL_HOURS + DT_HOURS, DT_HOURS)

# ---------- Particle classes (range-based) ----------
MP_CLASSES = ["Nano", "Micro", "Macro"]
SIZE_RANGES = {
    "Nano": "<1 µm (true nanoplastics)",
    "Micro": "1–100 µm",
    "Macro": "100 µm – 1–4 mm"
}
INITIAL = {"Nano": 1000.0, "Micro": 700.0, "Macro": 300.0}

# ---------- Compartments ----------
COMPARTMENTS = [
    "Nasal_Mucus",
    "Olfactory_Epithelium",
    "Olfactory_Bulb",
    "CSF",
    "Parenchyma",
    "Microglia",
    "Blood",
    "Passed_to_Gut",   # swallowed material (leaves this twin)
    "Cleared_Env"      # blown/wiped off / removed to environment
]

# ---------- Rate parameters per 0.5 h timestep (ASSUMED numeric values) ----------
# Rationale:
#  - Nano: highest chance to translocate by olfactory and cross into parenchyma/microglia.
#  - Micro: moderate deposition to epithelium; lower neuronal transport; higher mucus swallowing.
#  - Macro: mostly trapped / swallowed or cleared; negligible direct translocation to brain.
#
# These numbers are *assumptions* made to be biologically plausible (not exact literature per-30min rates).
RATES = {
    "Nano": {
        "nasal_to_epithelium": 0.12,    # fraction of nasal mucus moving into olfactory epithelium per step
        "epith_to_olfactory_bulb": 0.03,# neuronal transport from epithelium -> bulb (per step)
        "epith_to_csf": 0.01,          # peri-olfactory CSF route
        "olf_to_par": 0.02,            # bulb -> local parenchyma spread
        "nasal_swallow": 0.25,         # mucus clearance to gut per step
        "nasal_clear_env": 0.05,       # blown/wiped off per step
        "parenchyma_to_microglia": 0.03,# phagocytosis by microglia per step
        "microglia_release": 0.01,     # microglia release back to CSF per step
        "blood_to_parenchyma": 0.005,  # blood-borne crossing into parenchyma (small)
        "parenchyma_to_csf": 0.002,    # washout to CSF
        "csf_clear_swallow": 0.10      # CSF drainage fraction swallowed per step
    },
    "Micro": {
        "nasal_to_epithelium": 0.08,
        "epith_to_olfactory_bulb": 0.005,
        "epith_to_csf": 0.001,
        "olf_to_par": 0.01,
        "nasal_swallow": 0.35,
        "nasal_clear_env": 0.08,
        "parenchyma_to_microglia": 0.05,
        "microglia_release": 0.002,
        "blood_to_parenchyma": 0.0005,
        "parenchyma_to_csf": 0.0008,
        "csf_clear_swallow": 0.08
    },
    "Macro": {
        "nasal_to_epithelium": 0.03,
        "epith_to_olfactory_bulb": 0.0001,
        "epith_to_csf": 0.0,
        "olf_to_par": 0.005,
        "nasal_swallow": 0.50,
        "nasal_clear_env": 0.20,
        "parenchyma_to_microglia": 0.02,
        "microglia_release": 0.0002,
        "blood_to_parenchyma": 0.0,
        "parenchyma_to_csf": 0.0,
        "csf_clear_swallow": 0.05
    }
}

# ---------- Initialize state ----------
state = {ptype: {comp: 0.0 for comp in COMPARTMENTS} for ptype in MP_CLASSES}
for ptype in MP_CLASSES:
    # assume deposition at t=0 => start in Nasal_Mucus
    state[ptype]["Nasal_Mucus"] = float(INITIAL[ptype])

history = []  # snapshots saved *before* each step

# ---------- Simulation loop (deterministic, mass-conserving) ----------
for step in range(STEPS + 1):
    # Save snapshot
    history.append({ptype: {comp: state[ptype][comp] for comp in COMPARTMENTS} for ptype in MP_CLASSES})
    if step == STEPS:
        break

    new_state = {ptype: {comp: 0.0 for comp in COMPARTMENTS} for ptype in MP_CLASSES}

    for ptype in MP_CLASSES:
        r = RATES[ptype]
        s = state[ptype]

        # 1) Nasal mucus dynamics
        nasal = s["Nasal_Mucus"]
        to_epith = r["nasal_to_epithelium"] * nasal
        to_swallow = r["nasal_swallow"] * nasal
        to_clear_env = r["nasal_clear_env"] * nasal
        nasal_remain = nasal - (to_epith + to_swallow + to_clear_env)
        nasal_remain = max(0.0, nasal_remain)

        # 2) Olfactory epithelium: receives from nasal mucus
        epith = s["Olfactory_Epithelium"] + to_epith
        to_olf_bulb = r["epith_to_olfactory_bulb"] * epith
        to_csf_from_epith = r["epith_to_csf"] * epith
        epith_remain = epith - (to_olf_bulb + to_csf_from_epith)
        epith_remain = max(0.0, epith_remain)

        # 3) Olfactory bulb: gets neuronal transport
        olf = s["Olfactory_Bulb"] + to_olf_bulb
        olf_to_par = r["olf_to_par"] * olf
        olf_remain = olf - olf_to_par
        olf_remain = max(0.0, olf_remain)

        # 4) Parenchyma: receives from olfactory bulb + small blood-derived influx
        par = s["Parenchyma"] + olf_to_par
        from_blood = r["blood_to_parenchyma"] * s["Blood"]
        par += from_blood
        par_to_microglia = r["parenchyma_to_microglia"] * par
        par_to_csf = r["parenchyma_to_csf"] * par
        par_remain = par - (par_to_microglia + par_to_csf)
        par_remain = max(0.0, par_remain)

        # 5) Microglia: take up from parenchyma, some release back (to CSF)
        mg = s["Microglia"] + par_to_microglia
        mg_release = r["microglia_release"] * mg
        mg_remain = mg - mg_release
        mg_release = max(0.0, mg_release)
        mg_remain = max(0.0, mg_remain)

        # 6) CSF: receives from epithelium, parenchymal washout, and microglia release
        csf = s["CSF"] + to_csf_from_epith + par_to_csf + mg_release
        csf_to_swallow = r["csf_clear_swallow"] * csf
        csf_remain = csf - csf_to_swallow
        csf_remain = max(0.0, csf_remain)

        # 7) Blood: keep current blood (we do not model systemic inflows here beyond existing blood)
        blood = s["Blood"]  # mass-preserving placeholder

        # 8) Passed_to_Gut and Cleared_Env are cumulative
        passed_to_gut = s["Passed_to_Gut"] + to_swallow + csf_to_swallow
        cleared_env = s["Cleared_Env"] + to_clear_env

        # 9) Assign to new_state (everything nonnegative)
        new_state[ptype]["Nasal_Mucus"] = nasal_remain
        new_state[ptype]["Olfactory_Epithelium"] = epith_remain
        new_state[ptype]["Olfactory_Bulb"] = olf_remain
        new_state[ptype]["CSF"] = csf_remain
        new_state[ptype]["Parenchyma"] = par_remain
        new_state[ptype]["Microglia"] = mg_remain
        new_state[ptype]["Blood"] = blood
        new_state[ptype]["Passed_to_Gut"] = passed_to_gut
        new_state[ptype]["Cleared_Env"] = cleared_env

        # 10) Numerical conservation correction (tiny rounding errors)
        total_prev = sum(s[c] for c in COMPARTMENTS)
        total_now = sum(new_state[ptype][c] for c in COMPARTMENTS)
        diff = float(INITIAL[ptype]) - total_now
        if abs(diff) > 1e-9:
            # Put tiny residual back into Nasal_Mucus
            new_state[ptype]["Nasal_Mucus"] += diff
            # Ensure non-negative
            if new_state[ptype]["Nasal_Mucus"] < 0:
                rem = -new_state[ptype]["Nasal_Mucus"]
                new_state[ptype]["Nasal_Mucus"] = 0.0
                new_state[ptype]["Cleared_Env"] += rem

    # commit step
    state = new_state

# ---------- Build DataFrames for plotting ----------
dfs = {}
for ptype in MP_CLASSES:
    rows = []
    for h in history:
        rows.append({comp: h[ptype][comp] for comp in COMPARTMENTS})
    dfs[ptype] = pd.DataFrame(rows, index=TIMES)

# ---------- 1) PLOTS: show all compartment graphs first (one figure per compartment across types) ----------
for comp in COMPARTMENTS:
    plt.figure(figsize=(8,4))
    for ptype, color in zip(MP_CLASSES, ["blue","green","red"]):
        plt.plot(dfs[ptype].index, dfs[ptype][comp], label=f"{ptype} ({SIZE_RANGES[ptype]})", color=color)
    plt.title(f"{comp.replace('_',' ')} over time")
    plt.xlabel("Time (hours)")
    plt.ylabel("Number of particles")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# ---------- 2) Final numeric summary — same textual style as your other twins ----------
print("\n--- FINAL NUMERIC SUMMARY ---")
for ptype in MP_CLASSES:
    final = dfs[ptype].iloc[-1]
    initial = float(INITIAL[ptype])
    total = final.sum()
    print(f"\n{ptype} Particles (Initial {int(initial)}, {SIZE_RANGES[ptype]}):")
    for comp in COMPARTMENTS:
        val = final[comp]
        pct = (val / initial * 100.0) if initial > 0 else 0.0
        print(f"  {comp:20s}: {val:9.2f} ({pct:6.2f}%)")
    print(f"  Total conserved     : {total:9.2f} / {initial:.0f} ({(total/initial*100.0):6.2f}%)")

print("\nNervous-system twin simulation complete.")
